## 프로젝트: 멋진 작사가 만들기

####  1. 라이브러리 버전 확인

In [1]:
import glob # 사용자가 제시한 조건에 맞는 파일명을 리스트로 반환
import tensorflow

print(tensorflow.__version__)

2.11.0


#### 2. 데이터 읽어오기 

In [2]:
import glob
import os 

txt_path=os.getenv('Home')+'/aifell/node/221125_exploration_santence/lyrics/*'

txt_list=glob.glob(txt_path)
txt_list

['C:\\Users\\오순정/aifell/node/221125_exploration_santence/lyrics\\adele.txt',
 'C:\\Users\\오순정/aifell/node/221125_exploration_santence/lyrics\\al-green.txt',
 'C:\\Users\\오순정/aifell/node/221125_exploration_santence/lyrics\\alicia-keys.txt',
 'C:\\Users\\오순정/aifell/node/221125_exploration_santence/lyrics\\amy-winehouse.txt',
 'C:\\Users\\오순정/aifell/node/221125_exploration_santence/lyrics\\beatles.txt',
 'C:\\Users\\오순정/aifell/node/221125_exploration_santence/lyrics\\bieber.txt',
 'C:\\Users\\오순정/aifell/node/221125_exploration_santence/lyrics\\bjork.txt',
 'C:\\Users\\오순정/aifell/node/221125_exploration_santence/lyrics\\blink-182.txt',
 'C:\\Users\\오순정/aifell/node/221125_exploration_santence/lyrics\\bob-dylan.txt',
 'C:\\Users\\오순정/aifell/node/221125_exploration_santence/lyrics\\bob-marley.txt',
 'C:\\Users\\오순정/aifell/node/221125_exploration_santence/lyrics\\britney-spears.txt',
 'C:\\Users\\오순정/aifell/node/221125_exploration_santence/lyrics\\bruce-springsteen.txt',
 'C:\\Users\\오순정/aifel

In [3]:
raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r",encoding='UTF8') as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


#### 3. 데이터 정제

텍스트 파일을 확인해보니 가사 특성 상 연극 대사와 달리 끝에 반복적으로 오는 문자는 없었다

In [4]:
# enumerate() 함수를 이용하여 raw_corpus list 내에 저장된 문장과 그 문장의 인덱스를 반환(인덱스, 문장 순)
for idx,sentence in enumerate(raw_corpus):
    if len(sentence)==0: continue #길이가 0인문장건너뜀

    if idx> 20 : break
        
    print(sentence)

Looking for some education
Made my way into the night
All that bullshit conversation
Baby, can't you read the signs? I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love
If that's love in your eyes
It's more than enough
Had some bad love
So fast love is all that I've got on my mind Ooh, ooh
Ooh, ooh Looking for some affirmation
Made my way into the sun
My friends got their ladies
And they're all having babies
I just wanna have some fun I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love


문장 정제
- 특수 문자 제거 
- 모든 문자 소문자 변환 
- 앞 뒤 <start >, <end > 추가

In [5]:
import re #정규표현식 임포트

def preprocess_sentence(sentence):
    sentence=sentence.lower() #소문자화
    sentence=re.sub('[^a-zA-Z, ]', '', sentence)  # 공백, 알파벳,쉼표 제외한 특수문자 제거
    sentence=re.sub(r'["  "]+', " ", sentence)#공백 하나 이상인 곳 공백 제거
    sentence=sentence.strip() #양쪽 공백 제거
    sentence="<start>" +" "+sentence +" "+ "<end>" #앞 뒤 추가
    return sentence

print(preprocess_sentence("    G-L-O-R-I-A (Gloria) !  G-L-O-R-I-A    Gloria   "))

<start> gloria gloria gloria gloria <end>


위에서 구현한 함수로 정제된 문장 모음

In [6]:
corpus=[]

for sentence in raw_corpus:
    if len(sentence) == 0 : continue
        
    preprocessed_sentence=preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
    
#정제된 문장 확인
corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby, cant you read the signs i wont bore you with the details, baby <end>',
 '<start> i dont even wanna waste your time <end>',
 '<start> lets just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i aint mr right but if youre looking for fast love <end>',
 '<start> if thats love in your eyes <end>',
 '<start> its more than enough <end>']

토큰화

In [7]:
import tensorflow as tf

def tokenize(corpus):
    tokenizer=tf.keras.preprocessing.text.Tokenizer(
    num_words=12000, #담을 단어 만개로 제한 
    filters=' ', #정제하여 필터 하지 않음 
    oov_token="<unk>" #없는 단어는 엉크로 대체 
    )
    
    tokenizer.fit_on_texts(corpus) #문자 데이터를 입력 받아 리스트로 변환
    tensor=tokenizer.texts_to_sequences(corpus) #텍스트 안의 단어들을 연속된 숫자로 변환
    
    
    #토큰 갯수 15개 이상이묜 제외, <start>,<end> 포함해서 17
    tensor_t=[]
    
    for i in range(len(tensor)):
        if len(tensor[i])<=15:
            tensor_t.append(tensor[i])
              
    tensor_t=tf.keras.preprocessing.sequence.pad_sequences(tensor_t,padding='post') #입력 데이터 뒤에 패딩을 붙여 시퀀스 길이 맞춤 
    
    print(tensor_t,tokenizer)
    return tensor_t,tokenizer


tensor,tokenizer = tokenize(corpus)

[[  2 301  24 ...   0   0   0]
 [  2 212  10 ...   0   0   0]
 [  2  20  15 ...   0   0   0]
 ...
 [  2  23  69 ...   0   0   0]
 [  2  34  21 ...   0   0   0]
 [  2  23  69 ...   0   0   0]] <keras.preprocessing.text.Tokenizer object at 0x0000023FA6D870D0>


In [8]:
print(len(tensor))# tokenizer에 구축된 단어 사전 인덱스

163381


In [9]:
#단어 사전 인덱스 확인

for idx in tokenizer.index_word:
    print(idx,":",tokenizer.index_word[idx])
    
    if idx>=20 : break

1 : <unk>
2 : <start>
3 : <end>
4 : the
5 : i
6 : you
7 : and
8 : to
9 : a
10 : my
11 : me
12 : in
13 : it
14 : im
15 : that
16 : on
17 : of
18 : your
19 : like
20 : all


start 인덱스 넘버가 2여서 모두 2로 시작함을 알 수 있음 

#### 4. 평가 데이터셋 분리

데이터셋 생성

In [10]:
# 패딩으로 채우진 뒷 부분 <pad> 일 가능성 높음
src_input=tensor[:,:-1]
#tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input=tensor[:,1:]

print(src_input[0])
print(tgt_input[0])

[   2  301   24   91 6254    3    0    0    0    0    0    0    0    0]
[ 301   24   91 6254    3    0    0    0    0    0    0    0    0    0]


In [11]:
print(len(src_input))

163381


In [12]:
from sklearn.model_selection import train_test_split

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input,test_size=0.2,random_state=32,shuffle=True)

In [13]:
print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (130704, 14)
Target Train:  (130704, 14)


#### 5.인공지능 학습

In [14]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()

        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out

embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [15]:
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy( 
    from_logits=True, reduction='none') # 클래스 분류 문제에서 softmax 함수를 거치면 from_logits = False(default값),그렇지 않으면 from_logits = True.


model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
model.fit(enc_train, dec_train, epochs=10,batch_size=256,) # 만들어둔 데이터셋으로 모델을 학습한다. 30번 학습을 반복하겠다는 의미다.

Epoch 1/10
511/511 [==============================] - 3897s 8s/step - loss: 3.4049
Epoch 2/10
511/511 [==============================] - 3962s 8s/step - loss: 2.9921
Epoch 3/10
511/511 [==============================] - 3893s 8s/step - loss: 2.8247
Epoch 4/10
511/511 [==============================] - 3873s 8s/step - loss: 2.6944
Epoch 5/10
511/511 [==============================] - 3877s 8s/step - loss: 2.5862
Epoch 6/10
511/511 [==============================] - 3912s 8s/step - loss: 2.4903
Epoch 7/10
511/511 [==============================] - 3884s 8s/step - loss: 2.4014
Epoch 8/10
511/511 [==============================] - 3868s 8s/step - loss: 2.3186
Epoch 9/10
511/511 [==============================] - 3886s 8s/step - loss: 2.2402
Epoch 10/10
511/511 [==============================] - 3860s 8s/step - loss: 2.1660


In [26]:
model.summary()

Model: "text_generator"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  3072256   
                                                                 
 lstm (LSTM)                 multiple                  5246976   
                                                                 
 lstm_1 (LSTM)               multiple                  8392704   
                                                                 
 dense (Dense)               multiple                  12301025  
                                                                 
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


#### 6. 평가하기

In [18]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [21]:
generate_text(model, tokenizer, init_sentence="<start> i love ", max_len=20)
# generate_text 함수에 lyricist 라 정의한 모델을 이용해서 ilove 로 시작되는 문장을 생성

'<start> i love you <end> '

In [22]:
generate_text(model, tokenizer, init_sentence="<start> i eat ", max_len=20)

'<start> i eat my <unk> <end> '

In [29]:
generate_text(model, tokenizer, init_sentence="<start> fuck ", max_len=20)

'<start> fuck the fallout <end> '

In [30]:
generate_text(model, tokenizer, init_sentence="<start> she ", max_len=20)

'<start> she said she love me <end> '

욜

#### 회고

모델의 validation loss가 2.16으로 2.2 이하이긴 하다.
하지만 epoches를 10번만 돌린것이고, 파라미터를 수정도 해보지 않았다. 왜냐하면 이 모델을 학습시키는데 epoch
한번에 1시간이 걸렸기 때문이다... 10번 반복했데 저녁 9시에 돌리기 시작해서 잠자고 9시에 일어나니까 중간에 노트북이 
꺼졌던건지.. 여튼 거의 끝나있었다.. 시간이 너무 오래 걸려서 모델을 한번더 돌릴 엄두가 나지 않는다. 
왜 이렇게 오래 걸리지.. 

토큰화 과정에서 15 이하로 줄이기 위해 함수를 계속 보니 토큰화 과정의 텐서와 패딩에 대해 더 이해가 
잘 됬었다.